In [153]:
import numpy as np
import pandas as pd
#from textblob import TextBlob as tb
from time import time

In [154]:
df = pd.read_csv('../data/noticias_estadao.csv')

In [155]:
df = df.head(200)

# Meeting data

In [156]:
df.head()

,titulo,conteudo,idNoticia
0,PT espera 30 mil pessoas em festa na Esplanada,BRASÍLIA - Após o desgaste provocado com o lan...,1
1,Alckmin toma posse de olho no Planalto,"Reeleito em outubro, o governador tucano Geral...",2
2,Seis obstáculos e desafios do segundo mandato ...,1. Rearranjo das contas A nova equipe econôm...,3
3,Veja os desafios dos governadores que assumem ...,"No Acre, governador reeleito quer erradicar an...",4
4,PT impulsiona cerimônia de posse da Dilma nas ...,"Os perfis da presidente Dilma Rousseff, nas re...",5


In [157]:
print("Data has %d rows and %d columns" % df.shape)

Data has 200 rows and 3 columns


# Creating constants that will be used over this report

In [158]:
COLUMN_AXIS = 1
FULL_REPORT_COLNAME = 'noticia'
CONTENT_COLNAME = 'conteudo'
TITLE_COLNAME = 'titulo'
TOKENS_COLNAME = 'tokens'
TERM_COLNAME = 'term'
REPORT_ID_COLNAME = 'idNoticia'
AND = 'AND'
OR = 'OR'

# Concatenate alls reports' title and content in just one column.

In [159]:
def concatenate_report(df_row):
    """Concatenate report title and content in just one column.
        
        Args:
            df_row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            

        Return:
            str: full report (content with title) in lowercase.
    """

    full_report = df_row[TITLE_COLNAME] + " " + df_row[CONTENT_COLNAME]
    return full_report.lower()

In [160]:
df[FULL_REPORT_COLNAME] = df.apply(
    lambda row: concatenate_report(row), 
    axis=COLUMN_AXIS
)

Selecting just report's id and full content columns:

In [161]:
df = df[[REPORT_ID_COLNAME, FULL_REPORT_COLNAME]]

Dataframe now looks like:

In [162]:
df.head()

,idNoticia,noticia
0,1,pt espera 30 mil pessoas em festa na esplanada...
1,2,alckmin toma posse de olho no planalto reeleit...
2,3,seis obstáculos e desafios do segundo mandato ...
3,4,veja os desafios dos governadores que assumem ...
4,5,pt impulsiona cerimônia de posse da dilma nas ...


# Tokenizing report's text and saving tokens in another column in dataframe

In [163]:
def tokenize_text(df_row):
    """Tokenize the text content of a report given as a row from a DataFrame
        
        Args:
            df_row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            

        Return:
            set: a report content turned into a set of tokens.
    """    
    
    #text_blob = tb(df_row[FULL_REPORT_COLNAME]) 
    #m_tokens = set(text_blob.words)
    m_tokens = df_row[FULL_REPORT_COLNAME].split()
    return m_tokens

In [164]:
df[TOKENS_COLNAME] = df.apply(
    lambda row: tokenize_text(row), 
    axis=COLUMN_AXIS
)

# Creating inverted index

First, we will create a intermediate structure called unnested_tokens. This structure will save each token of a report, associating it to the report's id. After this step, we will group this unnested_tokens structure by tokens, getting all reports' ids in which a specific token appears.

In [165]:
def unnest_tokens_report(unnested_tokens_list, df_row):
    """Given a row observation of a DataFrame to represent a report (with content,
    tokens and id), iterate over the set of tokens and save each one as a dict with 
    token value and report id. Each dict is appended in the unnested_tokens_list
    passed as param.
        
        Args:
            unnested_tokens_list (list): list of dicts, each dict containing a token value 
                and the report id where it occured.
            df_row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            
    """  
    
    for token in df_row[TOKENS_COLNAME]:
        new_row = {
            TERM_COLNAME: token.strip('\'').strip(),
            REPORT_ID_COLNAME: df_row[REPORT_ID_COLNAME]
        }
        unnested_tokens_list.append(new_row)

In [166]:
unnested_tokens_list = []
df.apply(
    lambda row: unnest_tokens_report(unnested_tokens_list, row), 
    axis=COLUMN_AXIS
)

print("The unnested_tokens_list looks like: \n")
print(unnested_tokens_list[:9])
print("\nThe 'list of dicts' format will be used to create a pandas.DataFrame:")

The unnested_tokens_list looks like: 

[{'term': 'pt', 'idNoticia': 1}, {'term': 'espera', 'idNoticia': 1}, {'term': '30', 'idNoticia': 1}, {'term': 'mil', 'idNoticia': 1}, {'term': 'pessoas', 'idNoticia': 1}, {'term': 'em', 'idNoticia': 1}, {'term': 'festa', 'idNoticia': 1}, {'term': 'na', 'idNoticia': 1}, {'term': 'esplanada', 'idNoticia': 1}]

The 'list of dicts' format will be used to create a pandas.DataFrame:


In [167]:
unnested_tokens_df = pd.DataFrame(unnested_tokens_list)
unnested_tokens_df.head(10)

,idNoticia,term
0,1,pt
1,1,espera
2,1,30
3,1,mil
4,1,pessoas
5,1,em
6,1,festa
7,1,na
8,1,esplanada
9,1,brasília


### Grouping by term to create inverted index

In [168]:
class InvertedIndexTermOccurrence:
    """Class for register term frequency and docs' ids in which a 
    term of a inverted index structure appears.
    
    Attributes:
        term_freq (int): Quantity of docs in which term appears.
        docs_ids (list): ids of docs in which term appears.
    """
    
    def __init__(self, term_freq, docs_ids):
        self.term_freq = term_freq
        self.docs_ids = docs_ids
    
    def get_term_freq(self):
        return self.term_freq
    
    def get_docs_ids(self):
        return self.docs_ids
    
    def get_key(self):
        return self.term_freq

In [169]:
def create_inverted_index_structure(unnested_tokens_df):
    
    inverted_index = dict()

    for term, obs in unnested_tokens_df.groupby([TERM_COLNAME]):

        term_freq = len(obs.get_values())
        docs_ids = set(obs[REPORT_ID_COLNAME])

        inverted_index[term] = InvertedIndexTermOccurrence(term_freq, docs_ids)
    
    return inverted_index

In [170]:
inverted_index = create_inverted_index_structure(unnested_tokens_df)

## Processing Queries

In [171]:
def is_one_term_query(query):
    
    empty_str = ""
    space_str = " "
    
    if query == empty_str:
        raise ValueError('You should search for a non empty string.')
    
    elif query == space_str:
        return True
    
    else:
        return len(query.split(space_str)) == 1

In [172]:
def get_query_operator(query):
    return AND if AND in query else OR

In [214]:
def search_mul_terms(terms_to_search, operator):

    sorted_term_per_freq = list(map(lambda term: inverted_index[term], terms_to_search))    
    sorted(sorted_term_per_freq, key= lambda term: term.get_key())
    
    docs_ids_term1 = sorted_term_per_freq[0].get_docs_ids()   
    result = docs_ids_term1
    
    for another_term in sorted_term_per_freq:
        
        docs_ids = another_term.get_docs_ids()  
    
        if operator == AND:
            result = result & docs_ids
        elif operator == OR:
            result = result | docs_ids
            
    return list(result)

In [215]:
def search_two_terms(terms_to_search, operator):
        
    term1 = terms_to_search[0]
    term2 = terms_to_search[1]
    docs_ids_term1 = inverted_index[term1].get_docs_ids()
    docs_ids_term2 = inverted_index[term2].get_docs_ids()                
    
    if operator == AND:
        result = docs_ids_term1 & docs_ids_term2
    elif operator == OR:
        result = docs_ids_term1 | docs_ids_term2
        
    return list(result)

In [216]:
def search(query, inverted_index):
    
    if is_one_term_query(query):
        return inverted_index[query].get_docs_ids()  
    
    else:    
        operator = get_query_operator(query)
        terms_to_search = query.split(" " + operator + " ")
        terms_to_search = list(map(lambda term: term.lower(), terms_to_search)) # lowercase all terms to search
        
        return search_mul_terms(terms_to_search, operator)        

# Sanities checks

In [223]:
query = "pt AND espera AND pessoas"
search_result = search(query, inverted_index)
#correct_answer = sorted([1952, 4802, 1987, 6694, 5382, 1770, 2763, 1068, 5870, 2777, 1370, 2779])
#assert search_result == correct_answer

In [224]:
print(search_result)

{1, 129, 130, 4, 5, 6, 131, 138, 140, 15, 145, 146, 147, 148, 155, 159, 33, 176, 180, 181, 184, 59, 61, 64, 66, 197, 198, 72, 80, 93, 100, 106, 116, 117, 124, 125, 126, 127}


In [181]:
# inverted_index = dict()

# ROW = 0
# for row_index in range(df.shape[ROW]):
#     for token in df[TOKENS_COLNAME][row_index]:
        
#         report_id = df.at[row_index, REPORT_ID_COLNAME]
        
#         if token in inverted_index:
#             inverted_index[token].append(report_id)
#         else:
#             inverted_index[token] = [report_id]